In [0]:
!pip install pytorch_pretrained_bert
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('drive/Colab Notebooks')

In [0]:
# coding: UTF-8
import torch
import time 
import torch.nn as nn
import torch.nn.functional as F 
import pandas as pd 
import numpy as np 
from tqdm import tqdm 
from torch.utils.data import *
from pytorch_pretrained_bert import BertModel, BertTokenizer, BertConfig, BertAdam

In [0]:
tokenizer = BertTokenizer(vocab_file="data/vocab.txt")

In [0]:
input_ids = []
input_types = []
input_masks = []
pad_size = 64
 
with open("data/test.txt") as f:
    for i, l in tqdm(enumerate(f)): 
        x1, x2 = l.strip().split('\t')
        x1 = tokenizer.tokenize(x1)
        x2 = tokenizer.tokenize(x2)
        tokens = ["[CLS]"] + x1 + ["[SEP]"] + x2 +["[SEP]"]

        ids = tokenizer.convert_tokens_to_ids(tokens)
        types = [0] *(len(ids))
        masks = [1] * len(ids)
        if len(ids) < pad_size:
            types = types + [1] * (pad_size - len(ids))
            masks = masks + [0] * (pad_size - len(ids))
            ids = ids + [0] * (pad_size - len(ids))
        else:
            types = types[:pad_size]
            masks = masks[:pad_size]
            ids = ids[:pad_size]
        input_ids.append(ids)
        input_types.append(types)
        input_masks.append(masks)
        assert len(ids) == len(masks) == len(types) == pad_size

12500it [00:03, 4138.81it/s]


In [0]:
input_ids_train = np.array(input_ids)
input_types_train = np.array(input_types)
input_masks_train = np.array(input_masks)

BATCH_SIZE = 1
train_data = TensorDataset(torch.LongTensor(input_ids_train), 
                           torch.LongTensor(input_types_train), 
                           torch.LongTensor(input_masks_train))
 
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

In [0]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(path)
        for param in self.bert.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(768, 2)

    def forward(self, x):
        context = x[0]
        types = x[1]
        mask = x[2]
        _, pooled = self.bert(context, token_type_ids=types, 
                              attention_mask=mask, 
                              output_all_encoded_layers=False)
        out = self.fc(pooled)
        return out

In [0]:
pthpath = 'colab1.pth'
model = Model()
model = torch.load(pthpath)

In [0]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test(model, device, test_loader):
    res = []
    for batch_idx, (x1,x2,x3) in enumerate(test_loader):
        x1,x2,x3 = x1.to(device), x2.to(device), x3.to(device)
        with torch.no_grad():
            y_ = model([x1,x2,x3])
        pred = y_.max(-1, keepdim=True)[1]
        res.append(pred)
    return res

In [0]:
raw = test(model,DEVICE,train_loader)

In [0]:
result = []

for i in raw:
  result.append(i.cpu().item())

In [0]:
s = ''
for i in r:
    s = s + str(i) + '\n'

In [0]:
f = open('qaq.txt','w')
f.write(s)
f.close()